In [1]:
# save the final model to file
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
# define cnn model
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model
 
# run the test harness for evaluating a model
def run_test_harness():
# define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]
    # prepare iterator
    train_it = datagen.flow_from_directory('finalize_dogs_vs_cats/',
        class_mode='binary', batch_size=64, target_size=(224, 224))
    # fit model
    model.fit_generator(train_it, steps_per_epoch=len(train_it.filenames), epochs=1, verbose=1)
    # save model
    model.save('final_model.h5')
 
# entry point, run the test harness
run_test_harness()

Found 707 images belonging to 2 classes.
707/707 [==============================] - 57920s 82s/step - loss: 0.0983 - accuracy: 0.9920


In [4]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from os import listdir
from tqdm import tqdm
 
# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(224, 224))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 224, 224, 3)
    # center pixel data
    img = img.astype('float32')
    return img
 
# load an image and predict the class
def run_example(img,model):
    # load the image
    img = load_image(img)
    # load model
#     model = load_model('cnn_3layer_model.h5')
    model = model
    # predict the class
    result = model.predict(img)
    return result[0][0]

 
# evaluate model
src = 'photo/'
model = load_model('final_model.h5')
correct = 0
error = 0
for file in tqdm(listdir(src)):
    answer = run_example(src+file,model)
    if (file.startswith('cat') and answer == 0) or (file.startswith('dog') and answer == 1):
        correct += 1
    else:
        error += 1
print('correct - ', correct, 'error - ' ,error, 'accyracy -', int(correct/(correct+error)*100), '%')

100%|██████████| 24769/24769 [2:46:28<00:00,  3.07it/s]   

correct -  23133 error -  1636 accyracy - 93 %
